In [ ]:
from google.colab import files
files.upload()  # This will open a prompt to upload your kaggle.json file


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install kaggle


In [ ]:
!kaggle datasets list -s "chest x-ray"


In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia


In [ ]:
!kaggle datasets download -d praveengovi/coronahack-chest-xraydataset

In [ ]:
!kaggle datasets download -d tolgadincer/labeled-chest-xray-images

In [ ]:
!kaggle datasets download -d tawsifurrahman/tuberculosis-tb-chest-xray-dataset

In [ ]:
!kaggle datasets download -d kmader/pulmonary-chest-xray-abnormalities

In [ ]:
!unzip chest-xray-pneumonia.zip -d chest_xray_data


In [ ]:
!unzip coronahack-chest-xraydataset.zip -d chest_xray_data

In [ ]:
!unzip labeled-chest-xray-images.zip -d chest_xray_data

In [ ]:
!unzip tuberculosis-tb-chest-xray-dataset.zip -d chest_xray_data

In [ ]:
!unzip pulmonary-chest-xray-abnormalities.zip -d chest_xray_data

In [ ]:
# Imports
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import nltk
from nltk.tokenize import word_tokenize
import json


In [ ]:
import os
from glob import glob

def get_image_paths_and_labels(root_dir):
    data = []
    # ChinaSet
    china_normal = glob(os.path.join(root_dir, 'ChinaSet_AllFiles', 'ChinaSet_AllFiles', 'CXR_png', '*.png'))
    china_label = ['abnormal'] * len(china_normal)  # These may be labeled, update as needed

    # Coronahack
    corona_normal = glob(os.path.join(root_dir, 'Coronahack-Chest-XRay-Dataset', 'Coronahack-Chest-XRay-Dataset', 'Normal', '*.jpeg'))
    corona_abnormal = glob(os.path.join(root_dir, 'Coronahack-Chest-XRay-Dataset', 'Coronahack-Chest-XRay-Dataset', 'Viral Pneumonia', '*.jpeg'))
    corona_label_normal = ['normal'] * len(corona_normal)
    corona_label_abnormal = ['viral_pneumonia'] * len(corona_abnormal)

    # Montgomery
    montgomery_normal = glob(os.path.join(root_dir, 'Montgomery', 'MontgomerySet', 'Normal', '*.png'))
    montgomery_tub = glob(os.path.join(root_dir, 'Montgomery', 'MontgomerySet', 'Tuberculosis', '*.png'))
    montgomery_label_normal = ['normal'] * len(montgomery_normal)
    montgomery_label_tub = ['tuberculosis'] * len(montgomery_tub)

    # TB dataset
    tb_normal = glob(os.path.join(root_dir, 'TB_Chest_Radiography_Database', 'Normal', '*.png'))
    tb_tub = glob(os.path.join(root_dir, 'TB_Chest_Radiography_Database', 'Tuberculosis', '*.png'))
    tb_label_normal = ['normal'] * len(tb_normal)
    tb_label_tub = ['tuberculosis'] * len(tb_tub)

    # Aggregate
    paths = china_normal + corona_normal + corona_abnormal + montgomery_normal + montgomery_tub + tb_normal + tb_tub
    labels = china_label + corona_label_normal + corona_label_abnormal + montgomery_label_normal + montgomery_label_tub + tb_label_normal + tb_label_tub

    return paths, labels


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

# Label mapping
label_to_idx = {'normal':0, 'abnormal':1, 'viral_pneumonia':2, 'tuberculosis':3}

class ChestXrayFolderDataset(Dataset):
    def __init__(self, img_paths, img_labels, transform=None):
        self.img_paths = img_paths
        self.img_labels = img_labels
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
        ])

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")
        label = label_to_idx[self.img_labels[idx]]
        img = self.transform(img)
        return img, label


In [ ]:
# Get aggregated list
img_paths, img_labels = get_image_paths_and_labels('/content/chest_xray_data')

# Split into train/val/test (simple split below, can be randomized/proportional)
from sklearn.model_selection import train_test_split
train_imgs, val_imgs, train_lbls, val_lbls = train_test_split(img_paths, img_labels, test_size=0.2, random_state=42, stratify=img_labels)

# Build dataset and loader
train_dataset = ChestXrayFolderDataset(train_imgs, train_lbls)
val_dataset = ChestXrayFolderDataset(val_imgs, val_lbls)

from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Test batch
images, labels = next(iter(train_loader))
print("Batch image shape:", images.shape)
print("Batch label shape:", labels.shape)


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained VGG19 and adapt for multi-class
model = models.vgg19(pretrained=True)
for param in model.parameters():
    param.requires_grad = False   # Freeze all layers for feature extraction

# Replace the classifier for your labels
num_classes = len(label_to_idx)
model.classifier[6] = nn.Linear(4096, num_classes)
model = model.to(device)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=1e-4)

def train_one_epoch(loader, model, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / len(loader), correct / total

def validate(loader, model, criterion, device):
    model.eval()
    running_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / len(loader), correct / total


In [ ]:
# --- 1. Imports and Setup ---
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# For reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --- 2. Prepare File Paths, Labels, and Data Class ---

label_to_idx = {'normal':0, 'abnormal':1, 'viral_pneumonia':2, 'tuberculosis':3}

def get_image_paths_and_labels(root_dir):
    from glob import glob
    data = []
    # Add all paths and labels below; see earlier provided code
    # Example for demonstration:
    china_normal = glob(os.path.join(root_dir, 'ChinaSet_AllFiles', 'ChinaSet_AllFiles', 'CXR_png', '*.png'))
    china_label = ['abnormal'] * len(china_normal)
    corona_normal = glob(os.path.join(root_dir, 'Coronahack-Chest-XRay-Dataset', 'Coronahack-Chest-XRay-Dataset', 'Normal', '*.jpeg'))
    corona_abnormal = glob(os.path.join(root_dir, 'Coronahack-Chest-XRay-Dataset', 'Coronahack-Chest-XRay-Dataset', 'Viral Pneumonia', '*.jpeg'))
    corona_label_normal = ['normal'] * len(corona_normal)
    corona_label_abnormal = ['viral_pneumonia'] * len(corona_abnormal)
    tb_normal = glob(os.path.join(root_dir, 'TB_Chest_Radiography_Database', 'Normal', '*.png'))
    tb_tub = glob(os.path.join(root_dir, 'TB_Chest_Radiography_Database', 'Tuberculosis', '*.png'))
    tb_label_normal = ['normal'] * len(tb_normal)
    tb_label_tub = ['tuberculosis'] * len(tb_tub)
    # Aggregate
    paths = china_normal + corona_normal + corona_abnormal + tb_normal + tb_tub
    labels = china_label + corona_label_normal + corona_label_abnormal + tb_label_normal + tb_label_tub
    return paths, labels

class ChestXrayFolderDataset(Dataset):
    def __init__(self, img_paths, img_labels, transform=None):
        self.img_paths = img_paths
        self.img_labels = img_labels
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
        ])

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")
        label = label_to_idx[self.img_labels[idx]]
        img = self.transform(img)
        return img, label

# --- 3. Dataset Split and Loader ---

# Change this path to your actual dataset root!
root_dir = '/content/chest_xray_data'
img_paths, img_labels = get_image_paths_and_labels(root_dir)
train_imgs, val_imgs, train_lbls, val_lbls = train_test_split(img_paths, img_labels, test_size=0.2, random_state=42, stratify=img_labels)

train_dataset = ChestXrayFolderDataset(train_imgs, train_lbls)
val_dataset = ChestXrayFolderDataset(val_imgs, val_lbls)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Train images: {len(train_dataset)}, Val images: {len(val_dataset)}")

# --- 4. Model Definition ---

model = models.vgg19(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.classifier[6] = nn.Linear(4096, len(label_to_idx))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=1e-4)

# --- 5. Training and Validation Loop with Progress ---

def train_one_epoch(loader, model, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0, 0, 0
    num_batches = len(loader)
    for batch_idx, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        if (batch_idx+1) % 10 == 0 or (batch_idx+1) == num_batches:
            print(f"  [Batch {batch_idx+1}/{num_batches}] Train Loss: {loss.item():.4f} | Acc: {correct/total:.4f}")
    return running_loss / num_batches, correct / total

def validate(loader, model, criterion, device):
    model.eval()
    running_loss, correct, total = 0, 0, 0
    all_preds, all_labels = [], []
    num_batches = len(loader)
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            if (batch_idx+1) % 10 == 0 or (batch_idx+1) == num_batches:
                print(f"  [Batch {batch_idx+1}/{num_batches}] (Val) Loss: {loss.item():.4f} | Acc: {correct/total:.4f}")
    return running_loss / num_batches, correct / total, all_preds, all_labels

num_epochs = 5
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train_loss, train_acc = train_one_epoch(train_loader, model, criterion, optimizer, device)
    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    val_loss, val_acc, y_pred, y_true = validate(val_loader, model, criterion, device)
    print(f"[Epoch {epoch+1}] Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

# --- 6. Evaluate and Output Results ---

print("\nClassification Report (Validation Set):")
print(classification_report(y_true, y_pred, labels=list(label_to_idx.values()), target_names=list(label_to_idx.keys())))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# Save model for future use
torch.save(model.state_dict(), 'vgg19_chestxray_finetuned.pt')

# Visualize Sample Predictions
import matplotlib.pyplot as plt
class_idx_to_name = {idx: name for name, idx in label_to_idx.items()}

def show_sample_predictions(dataset, model, device, n=5):
    model.eval()
    plt.figure(figsize=(15,3))
    for i in range(n):
        img, label = dataset[i]
        with torch.no_grad():
            output = model(img.unsqueeze(0).to(device))
            pred = torch.argmax(output,1).item()
        plt.subplot(1,n,i+1)
        plt.imshow(img.permute(1,2,0).cpu().numpy()*0.229+0.485) # approx denormalize for visualization
        plt.title(f"True: {class_idx_to_name[label]}\nPred: {class_idx_to_name[pred]}")
        plt.axis('off')
    plt.show()

In [ ]:
# Manually extracted from your screenshots:
epochs = [1, 2, 3, 4, 5]
train_accs = [0.7580, 0.8629, 0.8827, 0.8971, 0.9033]
val_accs = [0.8602, 0.8880, 0.9054, 0.9260, 0.9260]
train_losses = [0.6136, 0.3084, 0.1675, 0.2696, 0.2514]
val_losses = [0.3965, 0.2999, 0.2657, 0.2352, 0.2175]



In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(epochs, train_accs, marker='o', label='Train Accuracy')
plt.plot(epochs, val_accs, marker='o', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

# Plot loss
plt.plot(epochs, train_losses, marker='o', label='Train Loss')
plt.plot(epochs, val_losses, marker='o', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd

df = pd.DataFrame({
    'Epoch': epochs,
    'Train Acc': train_accs,
    'Val Acc': val_accs,
    'Train Loss': train_losses,
    'Val Loss': val_losses
})

print(df.to_markdown(index=False))


In [ ]:
print("Best Validation Accuracy: {:.2f}% at epoch {}".format(
    max(val_accs)*100, epochs[val_accs.index(max(val_accs))]
))
print("Lowest Validation Loss: {:.4f} at epoch {}".format(
    min(val_losses), epochs[val_losses.index(min(val_losses))]
))


In [ ]:
import torch
from torchvision import models, transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2

# Utility for Grad-CAM
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hook_handles = []
        self._register_hooks()

    def _register_hooks(self):
        def backward_hook(module, grad_in, grad_out):
            # grad_out is a tuple, we are interested in the gradient w.r.t. the output of the module
            self.gradients = grad_out[0]

        def forward_hook(module, inp, out):
            if not out.requires_grad:
                # If the output doesn't require gradients (due to frozen layers),
                # clone it and explicitly make the clone require gradients.
                self.activations = out.clone().requires_grad_(True)
            else:
                self.activations = out
            self.activations.retain_grad()

        self.hook_handles.append(self.target_layer.register_forward_hook(forward_hook))
        self.hook_handles.append(self.target_layer.register_backward_hook(backward_hook))

    def remove_hooks(self):
        for handle in self.hook_handles:
            handle.remove()

    def __call__(self, image_tensor, class_idx=None):
        self.model.zero_grad()
        # Reset gradients and activations for each call
        self.gradients = None
        self.activations = None

        # Ensure the input tensor requires gradients for the backward pass
        if not image_tensor.requires_grad:
            image_tensor = image_tensor.clone().requires_grad_(True)

        output = self.model(image_tensor)

        if class_idx is None:
            class_idx = output.argmax(dim=1).item()

        class_score = output[0, class_idx]
        class_score.backward()

        # Check if hooks successfully captured data
        if self.activations is None:
            raise RuntimeError("Activations not captured. Ensure the forward hook is working.")
        if self.gradients is None:
            raise RuntimeError("Gradients not captured. Ensure the backward hook is working and gradients are flowing.")

        act = self.activations.detach().cpu().numpy()[0]
        grad = self.gradients.detach().cpu().numpy()[0]

        weights = np.mean(grad, axis=(1,2))
        cam = np.sum(weights[:, None, None] * act, axis=0)
        cam = np.maximum(cam, 0)

        # Normalize CAM to range [0, 1]
        if cam.max() == cam.min(): # Avoid division by zero if cam is all zeros
            cam = np.zeros_like(cam)
        else:
            cam = (cam - cam.min()) / (cam.max() - cam.min())

        cam = cv2.resize(cam, (224,224), interpolation=cv2.INTER_LINEAR)
        return cam

# Example usage for one validation image:
target_layer = model.features[-1]
gradcam = GradCAM(model, target_layer)
img, label = val_dataset[0]
input_tensor = img.unsqueeze(0).to(device)
cam = gradcam(input_tensor)
gradcam.remove_hooks()

# Display CAM
img_np = img.permute(1,2,0).cpu().numpy() * 0.229 + 0.485
# Clip the image values to [0, 1] to prevent the matplotlib warning
img_np = np.clip(img_np, 0, 1)
plt.imshow(img_np, cmap='gray')
plt.imshow(cam, cmap='jet', alpha=0.5)
plt.title("Grad-CAM overlay")
plt.axis('off')
plt.show()

In [ ]:
def forward_hook(module, inp, out):
    # Force requires_grad for the output, to ensure backward hooks work
    out.requires_grad_()
    self.activations = out


In [ ]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, inp, out):
            # Ensure output requires grad for Grad-CAM, even if model is frozen
            out.requires_grad_()
            self.activations = out
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0]
        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    def __call__(self, input_tensor, class_idx=None):
        self.model.zero_grad()
        output = self.model(input_tensor)
        if class_idx is None:
            class_idx = output.argmax(dim=1).item()
        class_score = output[0, class_idx]
        class_score.backward()
        activations = self.activations.detach().cpu().numpy()[0]
        gradients = self.gradients.detach().cpu().numpy()[0]
        weights = np.mean(gradients, axis=(1,2))
        cam = np.sum(weights[:, None, None] * activations, axis=0)
        cam = np.maximum(cam, 0)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-10)
        cam = cv2.resize(cam, (224,224))
        return cam


In [ ]:
import cv2

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hook_handles = [] # Added for clean hook management
        self._register_hooks()

    def _register_hooks(self):
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0]

        def forward_hook(module, inp, out):
            if not out.requires_grad:
                # If the output doesn't require gradients (due to frozen layers),
                # clone it and explicitly make the clone require gradients.
                self.activations = out.clone().requires_grad_(True)
            else:
                self.activations = out
            self.activations.retain_grad()

        self.hook_handles.append(self.target_layer.register_forward_hook(forward_hook))
        self.hook_handles.append(self.target_layer.register_backward_hook(backward_hook))

    def remove_hooks(self):
        for handle in self.hook_handles:
            handle.remove()

    def __call__(self, input_tensor, class_idx=None):
        self.model.zero_grad()
        # Reset gradients and activations for each call
        self.gradients = None
        self.activations = None

        # Ensure the input tensor requires gradients for the backward pass
        if not input_tensor.requires_grad:
            input_tensor = input_tensor.clone().requires_grad_(True)

        output = self.model(input_tensor)

        if class_idx is None:
            class_idx = output.argmax(dim=1).item()

        class_score = output[0, class_idx]
        class_score.backward(retain_graph=True)

        # Check if hooks successfully captured data
        if self.activations is None:
            raise RuntimeError("Activations not captured. Ensure the forward hook is working.")
        if self.gradients is None:
            raise RuntimeError("Gradients not captured. Ensure the backward hook is working and gradients are flowing.")

        activations = self.activations.detach().cpu().numpy()[0]
        gradients = self.gradients.detach().cpu().numpy()[0]
        weights = np.mean(gradients, axis=(1,2))
        cam = np.sum(weights[:, None, None] * activations, axis=0)
        cam = np.maximum(cam, 0)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-10)
        cam = cv2.resize(cam, (224,224))
        return cam

In [ ]:
target_layer = model.features[35]  # VGG19's last Conv2d
gradcam = GradCAM(model, target_layer)

img, label = val_dataset[0]
input_tensor = img.unsqueeze(0).to(device)
cam = gradcam(input_tensor)

img_np = img.permute(1,2,0).cpu().numpy()
img_np = np.clip(img_np*0.229 + 0.485, 0, 1)
plt.imshow(img_np, cmap='gray')
plt.imshow(cam, cmap='jet', alpha=0.5)
plt.title(f"Grad-CAM Overlay (True: {label})")
plt.axis('off')
plt.show()


In [ ]:
# Save
torch.save(model.state_dict(), 'vgg19_chestxray_classifier.pt')

# Load (for inference after restart)
model = torchvision.models.vgg19(pretrained=True)
model.classifier[6] = nn.Linear(4096, len(label_to_idx))
model.load_state_dict(torch.load('vgg19_chestxray_classifier.pt'))
model.eval()
model = model.to(device)


In [ ]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, inp, out):
            # Instead of retain_grad(), enable requires_grad on output
            out.requires_grad_()
            self.activations = out
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0]

        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    def __call__(self, input_tensor, class_idx=None):
        self.model.zero_grad()
        output = self.model(input_tensor)
        if class_idx is None:
            class_idx = output.argmax(dim=1).item()
        class_score = output[0, class_idx]
        class_score.backward()
        activations = self.activations.detach().cpu().numpy()[0]
        gradients = self.gradients.detach().cpu().numpy()[0]
        weights = gradients.mean(axis=(1,2))
        cam = (weights[:, None, None] * activations).sum(axis=0)
        cam = np.maximum(cam, 0)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-10)
        cam = cv2.resize(cam, (224, 224))
        return cam



In [ ]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None
        self.gradients = None
        self._register_forward_hook()

    def _register_forward_hook(self):
        def forward_hook(module, input, output):
            self.activations = output
        self.target_layer.register_forward_hook(forward_hook)

    def __call__(self, input_tensor, class_idx=None):
        self.model.eval()
        self.activations = None

        # Forward pass
        output = self.model(input_tensor)

        if class_idx is None:
            class_idx = output.argmax(dim=1).item()
        class_score = output[0, class_idx]

        # Backward pass w.r.t activations
        self.model.zero_grad()
        gradients = torch.autograd.grad(outputs=class_score, inputs=self.activations,
                                        grad_outputs=torch.ones_like(class_score),
                                        create_graph=True, retain_graph=True)[0]
        gradients = gradients.detach().cpu().numpy()[0]
        activations = self.activations.detach().cpu().numpy()[0]

        weights = np.mean(gradients, axis=(1, 2))
        cam = np.sum(weights[:, None, None] * activations, axis=0)

        cam = np.maximum(cam, 0)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-10)
        cam = cv2.resize(cam, (224, 224))
        return cam

# Usage
target_layer = model.features[35]  # last conv layer in VGG19
gradcam = GradCAM(model, target_layer)

img_tensor, label = val_dataset[0]
input_tensor = img_tensor.unsqueeze(0).to(device)
cam = gradcam(input_tensor)

img_np = img_tensor.permute(1, 2, 0).cpu().numpy()
img_np = np.clip(img_np * 0.229 + 0.485, 0, 1)

plt.imshow(img_np, cmap='gray')
plt.imshow(cam, cmap='jet', alpha=0.5)
plt.title(f"Grad-CAM Overlay (True label: {label})")
plt.axis('off')
plt.show()


